In [3]:
import pandas as pd
import dill as pickle
import requests
import json
import warnings
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

pd.set_option('display.max_columns', 50)

### Конвейер обучения и применения модели через Pipeline

Подгружаем данные [отсюда](https://www.kaggle.com/datasets/abcsds/pokemon) и изучаем их 

In [4]:
data = pd.read_csv('Pokemon.csv')
data.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800 entries, 0 to 799
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   #           800 non-null    int64 
 1   Name        800 non-null    object
 2   Type 1      800 non-null    object
 3   Type 2      414 non-null    object
 4   Total       800 non-null    int64 
 5   HP          800 non-null    int64 
 6   Attack      800 non-null    int64 
 7   Defense     800 non-null    int64 
 8   Sp. Atk     800 non-null    int64 
 9   Sp. Def     800 non-null    int64 
 10  Speed       800 non-null    int64 
 11  Generation  800 non-null    int64 
 12  Legendary   800 non-null    bool  
dtypes: bool(1), int64(9), object(3)
memory usage: 75.9+ KB


Будем прогнозировать значение переменной Legendary по остальным признакам, приведем его в числовому виду.  
Поле # и имя нам не понадобятся.  
Также удаляем поле Total, поскольку оно выражается через другие поля.  

In [6]:
data.drop(['#', 'Name', 'Total'], axis=True, inplace=True)
data.head()

,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,Grass,Poison,45,49,49,65,65,45,1,False
1,Grass,Poison,60,62,63,80,80,60,1,False
2,Grass,Poison,80,82,83,100,100,80,1,False
3,Grass,Poison,80,100,123,122,120,80,1,False
4,Fire,NaN,39,52,43,60,50,65,1,False


Отделяем признаки от целевой переменной.

In [7]:
features = data.drop(['Legendary'], axis=1)
target = 1 * data['Legendary']

Классы сбалансированны?

In [8]:
target.value_counts()

0    735
1     65
Name: Legendary, dtype: int64

Отделим обучающую часть от тестовой

In [9]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.3, random_state=43)

Посмотрим на значения категориальных переменных

In [10]:
features_train['Type 1'].unique()

array(['Water', 'Normal', 'Fire', 'Dark', 'Grass', 'Bug', 'Fighting',
       'Ground', 'Dragon', 'Ice', 'Poison', 'Fairy', 'Steel', 'Ghost',
       'Psychic', 'Electric', 'Flying', 'Rock'], dtype=object)

In [11]:
features_train['Type 2'].unique()

array([nan, 'Flying', 'Normal', 'Dragon', 'Psychic', 'Poison', 'Ground',
       'Dark', 'Fighting', 'Water', 'Electric', 'Fire', 'Ice', 'Ghost',
       'Steel', 'Fairy', 'Rock', 'Grass', 'Bug'], dtype=object)

In [12]:
types_all = sorted(set(pd.concat([features_train['Type 1'], features_train['Type 2']]).dropna()))

Развернем типы во флаговые колонки: для каждого типа будет колонка, в которой значение 1 будет, если такой тип у покемона есть, а иначе - значение 0

In [13]:
class PokemonTypesToFlags(TransformerMixin, BaseEstimator):

    def __init__(self):
        self.types_all = types_all
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.reset_index(drop=True)
        
        ohe_types = pd.DataFrame.from_dict(
            list(
                X
                .apply(
                    lambda f: dict([
                        (
                            t, 1 if f['Type 1'] == t or f['Type 2'] == t else 0
                        )
                        for t in self.types_all
                    ]),
                    axis=1
                )
            )
        )
        
        X = pd.concat([X.drop(['Type 1', 'Type 2'], axis=1), ohe_types], axis=1)
        return X

Преобразуем тренировочный датасет 

In [14]:
pttf = PokemonTypesToFlags()
pttf.fit(features_train)
features_train_pptf = pttf.transform(features_train)
features_train_pptf.head()

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,80,80,80,80,80,80,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,40,55,30,30,30,60,4,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0
2,79,103,120,135,115,78,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,62,50,58,73,54,72,6,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0
4,90,120,100,150,120,100,4,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


Отмасштабируем признаки

In [15]:
scaler = StandardScaler()
scaler.fit(features_train_pptf)
features_train_pptf_scaled = pd.DataFrame(scaler.transform(features_train_pptf), columns=scaler.feature_names_in_)
features_train_pptf_scaled.head()

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Bug,Dark,Dragon,Electric,Fairy,Fighting,Fire,Flying,Ghost,Grass,Ground,Ice,Normal,Poison,Psychic,Rock,Steel,Water
0,0.456275,0.039980,0.199806,0.223015,0.338148,0.404960,0.424085,-0.330017,-0.237915,-0.233696,-0.258199,-0.237915,-0.269809,-0.309662,-0.377964,-0.242077,-0.377964,-0.306186,-0.211604,-0.390209,-0.302685,-0.349635,-0.258199,-0.254242,2.300464
1,-1.179014,-0.741748,-1.387753,-1.323398,-1.588623,-0.281932,0.424085,-0.330017,-0.237915,-0.233696,-0.258199,-0.237915,-0.269809,-0.309662,2.645751,-0.242077,-0.377964,-0.306186,-0.211604,2.562727,-0.302685,-0.349635,-0.258199,-0.254242,-0.434695
2,0.415392,0.759170,1.469853,1.924069,1.686888,0.336270,-1.361535,-0.330017,-0.237915,-0.233696,-0.258199,-0.237915,-0.269809,-0.309662,-0.377964,-0.242077,-0.377964,-0.306186,-0.211604,-0.390209,-0.302685,-0.349635,-0.258199,-0.254242,2.300464
3,-0.279605,-0.898094,-0.498720,0.006517,-0.663773,0.130203,1.614497,-0.330017,-0.237915,-0.233696,-0.258199,-0.237915,-0.269809,3.229330,-0.377964,-0.242077,-0.377964,-0.306186,-0.211604,2.562727,-0.302685,-0.349635,-0.258199,-0.254242,-0.434695
4,0.865097,1.290745,0.834829,2.387992,1.879566,1.091852,0.424085,-0.330017,-0.237915,4.279059,-0.258199,-0.237915,-0.269809,-0.309662,-0.377964,-0.242077,-0.377964,-0.306186,-0.211604,-0.390209,-0.302685,-0.349635,-0.258199,-0.254242,2.300464


Построим простой классификатор с помощью полученных признаков и оценим его качество

In [16]:
model = DecisionTreeClassifier(random_state=0, class_weight='balanced')
model.fit(features_train_pptf_scaled, target_train)
print('Качество модели на обучающей выборке:', {roc_auc_score(target_train, model.predict_proba(features_train_pptf_scaled)[:, 1])})

Качество модели на обучающей выборке: {1.0}


Подготовим конвейнер, чтобы можно было запустить все эти трансформеры последовательно одной командой

In [17]:
pipe = Pipeline([  
    ('ohe_types', PokemonTypesToFlags()),
    ('scaler', StandardScaler()),
    ('classify', DecisionTreeClassifier(class_weight='balanced', random_state=0))
])

Настраиваем трансформеры однократно на исходной обучающей выборке и проверяем, что они работают, как ожидается.

In [18]:
pipe.fit(X=features_train, y=target_train)
print('Качество модели на обучающей выборке:', {roc_auc_score(target_train, pipe.predict_proba(features_train)[:, 1])})

Качество модели на обучающей выборке: {1.0}


Проверим качество на тестовой части

In [19]:
print('Качество модели на тестовой выборке:', {roc_auc_score(target_test, pipe.predict_proba(features_test)[:, 1])})

Качество модели на тестовой выборке: {0.7028701891715591}


Модель явно переобучилась. А какая у дерева глубина?

In [20]:
pipe['classify'].tree_.max_depth

20

Переберем несколько вариантов, с помощью кросс-валидации, чтобы улучшить качество на тестовой выборке

In [21]:
pipe = Pipeline([  
  ('ohe_types', PokemonTypesToFlags()),
  ('scaler', StandardScaler()),
  ('classify', DecisionTreeClassifier(class_weight='balanced', random_state=0))
])

params = [
    {'classify': [LogisticRegression(class_weight='balanced', random_state=0)]}, 
    {'classify': [DecisionTreeClassifier(class_weight='balanced', random_state=0)], 'classify__max_depth': [2, 5, 10, 20]}
]

grid_search = GridSearchCV(pipe, param_grid=params, cv=5, scoring='roc_auc')
grid_search.fit(X=features_train, y=target_train)
print(
    'Качество модели на тестовой выборке c лучшей моделью:', 
    {roc_auc_score(target_test, grid_search.predict_proba(features_test)[:, 1])}
)

Качество модели на тестовой выборке c лучшей моделью: {0.989997825614264}


In [22]:
grid_search.best_estimator_

Pipeline(steps=[('ohe_types', PokemonTypesToFlags()),
                ('scaler', StandardScaler()),
                ('classify',
                 LogisticRegression(class_weight='balanced', random_state=0))])

In [23]:
pd.DataFrame.from_dict(grid_search.cv_results_).transpose()

,0,1,2,3,4
mean_fit_time,3.705251,0.04096,0.035826,0.036995,0.035155
std_fit_time,0.775232,0.009442,0.000128,0.002966,0.000412
mean_score_time,0.180574,0.010915,0.011107,0.011598,0.010537
std_score_time,0.0495,0.000254,0.0006,0.0019,0.00021
param_classify,"LogisticRegression(class_weight='balanced', ra...",DecisionTreeClassifier(class_weight='balanced'...,DecisionTreeClassifier(class_weight='balanced'...,DecisionTreeClassifier(class_weight='balanced'...,DecisionTreeClassifier(class_weight='balanced'...
param_classify__max_depth,NaN,2,5,10,20
params,{'classify': LogisticRegression(class_weight='...,{'classify': DecisionTreeClassifier(class_weig...,{'classify': DecisionTreeClassifier(class_weig...,{'classify': DecisionTreeClassifier(class_weig...,{'classify': DecisionTreeClassifier(class_weig...
split0_test_score,0.985577,0.874399,0.891226,0.895433,0.778846
split1_test_score,0.952535,0.796117,0.687702,0.657497,0.661812
split2_test_score,0.985976,0.772384,0.816613,0.756203,0.75836


In [26]:
grid_search.best_estimator_

Pipeline(steps=[('ohe_types', PokemonTypesToFlags()),
                ('scaler', StandardScaler()),
                ('classify',
                 LogisticRegression(class_weight='balanced', random_state=0))])

### Оборачивание модели в сервис API

Сериализуем (консервируем) модель

In [27]:
with open('./models/best_pokemon_model.pk', 'wb') as file:
    pickle.dump(grid_search, file)

Десериализуем ее, чтобы убедиться, что она правильно работает

In [28]:
with open('./models/best_pokemon_model.pk','rb') as f:
    loaded_model = pickle.load(f)

In [31]:
print(
    'Качество модели на тестовой выборке от законсервированной модели:', 
    {roc_auc_score(target_test, loaded_model.predict_proba(features_test)[:, 1])}
)

Качество модели на тестовой выборке от законсервированной модели: {0.989997825614264}


Функцию-обертку реализуем в отдельном файле best_pokemon_service.py

Чтобы запустить API, нужно в терминале перейти в папку с кодом сервиса и ввести ```gunicorn --bind 0.0.0.0:8000 best_pokemon_service:app```  
Если ошибка ```Connection in use: ('0.0.0.0', 8000)```
- то либо делаем ```kill <номер, который у процесса, который занял порт>```
- либо пробуем вместо 8000 другой

После запуска API, можно им пользоваться 

In [34]:
features_test

,Type 1,Type 2,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation
586,Psychic,Flying,55,45,43,55,43,72,5
766,Rock,Dragon,58,89,77,45,45,48,6
722,Fire,NaN,59,59,58,90,70,73,6
580,Normal,Flying,80,115,80,65,55,93,5
542,Fire,Steel,91,90,106,130,106,77,4
...,...,...,...,...,...,...,...,...,...
525,Normal,NaN,85,80,70,135,75,90,4
338,Electric,NaN,70,75,60,105,60,105,3
606,Grass,Fairy,40,27,60,37,50,66,5
596,Water,Ground,75,65,55,65,55,69,5


In [41]:
header = {
    'Content-Type': 'application/json',
    'Accept': 'application/json'
}

resp = requests.post(
    "http://0.0.0.0:6000/predict", 
    data=json.dumps(features_test.to_json(orient='records')),
    headers=header
)

In [42]:
resp.status_code

200

In [46]:
print(
    'Качество модели на тестовой выборке от модели в API:', 
    {roc_auc_score(target_test, pd.read_json(resp.json()['predictions']))}
)

Качество модели на тестовой выборке от модели в API: {0.989997825614264}
